In [15]:
# confident edges: edges that iterative approach predicted early (in other words, "confidently")
# we compare our method and LR method on the accuracy of those edges


import _pickle as pkl
import networkx as nx
import numpy as np
import random

from tqdm import tqdm
from snpp.cores.lowrank import alq_spark, predict_signs
from snpp.utils.matrix import split_train_test, load_sparse_csr
from snpp.utils.signed_graph import g2m
from snpp.utils.data import load_train_test_graphs
from snpp.utils.edge_filter import filter_by_min_triangle_count

from snpp.utils.spark import sc

dataset = 'slashdot'
lambda_ = 0.2
k = 10
max_iter = 100
random_seed = 123456
min_tri_count = 30

recache_input = False

random.seed(random_seed)
np.random.seed(random_seed)

In [2]:
train_g, test_g = load_train_test_graphs(dataset, recache_input)


loading train and test graphs...


In [3]:
################
## DEBUG
#################
print(train_g.number_of_nodes())
print(train_g.number_of_edges())
print(test_g.number_of_edges())
assert train_g.number_of_nodes() == test_g.number_of_nodes()

77357
464917
51658


In [4]:
train_g_ud = train_g.to_undirected()

train_m = g2m(train_g)
train_m_ud = g2m(train_g_ud)
print(train_m_ud.shape)
print(train_m_ud.nnz)

100%|██████████| 77357/77357 [00:01<00:00, 58879.16it/s]


(77357, 77357)
929435


In [17]:
confident_edges = set(filter_by_min_triangle_count(train_g_ud, test_g.edges(), min_tri_count))
print(len(confident_edges))

1819


In [12]:
X, Y = alq_spark(train_m, k=k, sc=sc,
                 lambda_=lambda_, iterations=max_iter,
                 seed=random_seed)

KeyboardInterrupt: 

In [18]:
truth = set((i, j, test_g[i][j]['sign']) for i, j in confident_edges)
preds = predict_signs(X, Y, confident_edges, sc)
assert len(truth) == len(preds)
assert set((i, j) for i, j, _ in preds) == set((i ,j) for i, j, _ in truth)

print('=> accuracy on {} edges {} (lowrank)'.format(len(preds), len(truth.intersection(preds)) / len(truth)))

=> accuracy on 1819 edges 0.9835074216602528 (lowrank)


In [ ]:
## Iterative approach
from snpp.cores.joint_part_pred import iterative_approach
from snpp.cores.max_balance import faster_greedy
from snpp.cores.lowrank import partition_graph
from snpp.cores.budget_allocation import constant_budget
from snpp.cores.triangle import build_edge2edges

part, iter_preds, status = iterative_approach(
    train_g_ud,
    T=confident_edges,
    k=k,
    graph_partition_f=partition_graph,
    graph_partition_kwargs=dict(sc=sc,
                                lambda_=lambda_,
                                iterations=max_iter,
                                seed=random_seed),
    budget_allocation_f=constant_budget,
    budget_allocation_kwargs=dict(const=200),
    solve_maxbalance_f=faster_greedy,
    solve_maxbalance_kwargs={'edge2edges': build_edge2edges(train_g_ud.copy(),
                                                            confident_edges)},
    truth=set([(i, j, test_g[i][j]['sign'])
               for i, j in confident_edges]),
    perform_last_partition=False)

# Accuracy on 1819 predictions is 0.9835074216602528
# => accuracy 0.9835074216602528 (strong balance)


 21%|██        | 373/1819 [00:00<00:00, 1878.81it/s]

build edge2edges


100%|██████████| 1819/1819 [00:00<00:00, 1843.61it/s]


iteration=1, #remaining targets=1819
graph partitioning...
to_scipy_sparse_matrix


100%|██████████| 77357/77357 [00:01<00:00, 56537.59it/s]


...

ALS...
predict labels (SVD + Kmeans)...
eigen values: [ 192.50225274   94.314556     74.98190964   56.35275535   46.55712091
   42.81516672   42.09468087   41.59070258   40.387221     36.23217826]


/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T

solving max_balance
building triangle_count_by_edge
edge2edges (size 1714) is given
✓ assigning -1 to (1491, 1240) produces 34 more balanced triangles (42, 8): (('s-1', 34), ('w-1', 8), ('s+1', 8))
✓ assigning -1 to (1325, 676) produces 33 more balanced triangles (36, 3): (('s-1', 34), ('s+1', 3), ('w-1', 2))
✓ assigning 1 to (1234, 1988) produces 32 more balanced triangles (0, 32): (('s+1', 32),)
✓ assigning 1 to (1792, 6706) produces 32 more balanced triangles (0, 32): (('s+1', 32),)
✓ assigning 1 to (9773, 1029) produces 32 more balanced triangles (1, 33): (('s+1', 33), ('s-1', 1))
✓ assigning 1 to (74, 1550) produces 32 more balanced triangles (2, 34): (('s+1', 34), ('s-1', 2))
✓ assigning 1 to (2446, 190) produces 32 more balanced triangles (0, 32): (('s+1', 32),)
✓ assigning 1 to (659, 1029) produces 32 more balanced triangles (6, 38): (('s+1', 38), ('s-1', 4), ('w-1', 2))
✓ assigning 1 to (2384, 896) produces 32 more balanced triangles (1, 33): (('s+1', 33), ('s-1', 1))
✓ assign

100%|██████████| 77357/77357 [00:01<00:00, 57668.32it/s]


ALS...


In [9]:
## Confusion matrix
from sklearn.metrics import confusion_matrix

true_signs = [s for _, _, s in sorted(list(truth))] 
pred_signs = [s for _, _, s in sorted(list(preds))] 
print(confusion_matrix(true_signs, pred_signs, labels=[-1, 1]))

[[ 149   46]
 [  16 2434]]


In [ ]:
# Conclusion:
# Strong balance is more reliable than weak balance
# however, is it because of the poor quality of partitioning?